# Lesson 3 - Calling an A2A Agent using an A2A Client

In this lesson, you will use the A2A Python Client to interact with the Policy Agent you created and activated in the previous lesson. The client handles the protocol details, allowing you to discover the agent's capabilities via its `AgentCard` and send messages to it.

In [12]:
!uv pip show a2a-sdk


Name: a2a-sdk
Version: 0.3.16
Location: /Users/loinguyen/A2A/A2A_learning/.venv/lib/python3.12/site-packages
Requires: google-api-core, httpx, httpx-sse, protobuf, pydantic
Required-by: agent-framework-a2a, langgraph-a2a-server


In [6]:
import os

import httpx
from IPython.display import Markdown, display
from a2a.client import (
    Client,
    ClientConfig,
    ClientFactory,
    create_text_message_object,
)
from a2a.types import Artifact, Message, Task
from a2a.utils.message import get_message_text

from helpers import display_agent_card, setup_env

## 3.1. Check Server Status

First, ensure that your `PolicyAgent` server is running.
- Open a terminal as instructed below.
- If the agent is still running from the previous lesson, you don't need to do anything.
- If the agent has stopped, type: `uv run a2a_policy_agent.py`

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open a new terminal window in your Jupyter environment to run the server.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 3.2. Setup Client Configuration

Load environment variables to get the host and port of the running agent. You will construct the connection URL from these variables.


In [7]:
setup_env()

host = os.getenv("AGENT_HOST", "localhost")
port = os.getenv("POLICY_AGENT_PORT")
# port = os.getenv("RESEARCH_AGENT_PORT")

In [8]:
prompt = "How much would I pay for mental health therapy?"
# prompt = "How do I get mental health therapy?"

## 3.3. Run the Client Interaction

Now you will execute the client interaction flow:
1.  **Connect**: Establish a connection to the agent using `ClientFactory`.
2.  **Discover**: Retrieve the `AgentCard` using `get_card()`.
3.  **Construct**: Create a message object with the user's prompt.
4.  **Send & Receive**: Send the message and asynchronously process the response stream (which may include Tasks, Artifacts, or Messages).


In [10]:
async with httpx.AsyncClient(timeout=100.0) as httpx_client:
    # Step 1: Create a client
    client: Client = await ClientFactory.connect(
        f"http://{host}:{port}",
        client_config=ClientConfig(
            httpx_client=httpx_client,
        ),
    )

    # Step 2: Discover the agent by fetching its card
    agent_card = await client.get_card()
    display_agent_card(agent_card)

    # Step 3: Create the message using a convenient helper function
    message = create_text_message_object(content=prompt)

    display(Markdown(f"**Sending prompt:** `{prompt}` to the agent..."))

    # Step 4: Send the message and await the final response.
    responses = client.send_message(message)

    text_content = ""

    # Step 5: Process the responses from the agent
    async for response in responses:
        if isinstance(response, Message):
            # The agent replied directly with a final message
            print(f"Message ID: {response.message_id}")
            text_content = get_message_text(response)
        # response is a ClientEvent
        elif isinstance(response, tuple):
            task: Task = response[0]
            print(f"Task ID: {task.id}")
            if task.artifacts:
                artifact: Artifact = task.artifacts[0]
                print(f"Artifact ID: {artifact.artifact_id}")
                text_content = get_message_text(artifact)

    display(Markdown("### Final Agent Response\n-----"))
    if text_content:
        display(Markdown(text_content))
    else:
        display(
            Markdown(
                "**No final text content received or task did not complete successfully.**"
            )
        )

### Agent Card Details
| Property | Value |
| :--- | :--- |
| **Name** | InsurancePolicyCoverageAgent |
| **Description** | Provides information about insurance policy coverage options and details. |
| **Version** | `1.0.0` |
| **URL** | [http://localhost:9999/](http://localhost:9999/) |
| **Protocol Version** | `0.3.0` |

#### Skills
| Name | Description | Examples |
| :--- | :--- | :--- |
| **Insurance coverage** | Provides information about insurance coverage options and details. | • What does my policy cover?<br>• Are mental health services included? |

**Sending prompt:** `How much would I pay for mental health therapy?` to the agent...

Message ID: e63269e4-0fc1-440b-ab33-ec9a92fa6cc9


### Final Agent Response
-----

Based on the Summary of Benefits and Coverage, your payment for mental health therapy (outpatient services) depends on whether you use an in-network or out-of-network provider and if you have met your deductible:

*   **In-Network Provider:** You pay **10% coinsurance** after your deductible is met.
*   **Out-of-Network Provider:** You pay **30% coinsurance** after your deductible is met.

**Important Deductible Information:**
Generally, you must pay all costs out-of-pocket until you reach the overall deductible:
*   **In-Network:** \$1,700 for individuals / \$3,400 for families.
*   **Out-of-Network:** \$3,400 for individuals / \$6,800 for families.

## 3.4. Resources

- [A2A Python SDK Documentation](https://a2a-protocol.org/latest/sdk/python/api/)
